In [3]:
from pyspark.sql import SparkSession, functions as f, types as t, Column, DataFrame
from pyspark.sql.window import Window

spark = SparkSession.builder.getOrCreate()

# pics_data = spark.read.parquet('gs://genetics_etl_python_playground/XX.XX/output/python_etl/parquet/pics_credible_set/').persist()
# pics_data.filter(f.col('tagVariantId').isNotNull() & f.col('pics_99_perc_credset')).show()

# ld_data = spark.read.parquet('gs://genetics_etl_python_playground/XX.XX/output/python_etl/parquet/ld_expanded_assoc')
# ld_data.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/12/16 11:53:39 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/12/16 11:53:40 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/12/16 11:53:40 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/12/16 11:53:40 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


## Negative overall R

- This most likely can be explained by duplication... 

In [3]:
(
    pics_data
    .filter(
        (f.col('studyId') =='GCST000278_1') & 
        (f.col('variantId') =='1_112770941_C_A') &
        (f.col('tagVariantId') =='1_112779261_T_A')
    )
    .drop('chromosome')
    .show() 
)

+------------+---------------+---------------+---------+--------------------+-------------+--------------------+--------------------+
|     studyId|      variantId|   tagVariantId|R_overall|             pics_mu|pics_postprob|pics_95_perc_credset|pics_99_perc_credset|
+------------+---------------+---------------+---------+--------------------+-------------+--------------------+--------------------+
|GCST000278_1|1_112770941_C_A|1_112779261_T_A|-0.051554|0.014678789508465448|          NaN|                true|                true|
+------------+---------------+---------------+---------+--------------------+-------------+--------------------+--------------------+



In [4]:
(
    ld_data
    .filter(
        (f.col('studyId') =='GCST000278_1') & 
        (f.col('variantId') =='1_112770941_C_A') &
        (f.col('tagVariantId') =='1_112779261_T_A')
    )
    # .distinct()
    .select('variantId','gnomadPopulation','studyId','sampleSize','relativeSampleSize', 'tagVariantId', 'r')
    .show() 
)

+---------------+----------------+------------+----------+------------------+---------------+------------------+
|      variantId|gnomadPopulation|     studyId|sampleSize|relativeSampleSize|   tagVariantId|                 r|
+---------------+----------------+------------+----------+------------------+---------------+------------------+
|1_112770941_C_A|             nfe|GCST000278_1|    2790.0|               1.0|1_112779261_T_A|0.9745720726356035|
|1_112770941_C_A|             nfe|GCST000278_1|    2790.0|               1.0|1_112779261_T_A|0.9745720726356035|
|1_112770941_C_A|             nfe|GCST000278_1|    2790.0|               1.0|1_112779261_T_A|0.9745720726356035|
|1_112770941_C_A|             nfe|GCST000278_1|    2790.0|               1.0|1_112779261_T_A|0.9745720726356035|
+---------------+----------------+------------+----------+------------------+---------------+------------------+



## posterior probability NAN

In [5]:
#  1|  GCST000175|1_184051811_G_A|  1_184041593_C_A| 0.966352|   7.561187580349171|                 NaN|                true|                true|

(
    pics_data
    .filter(
        (f.col('studyId') =='GCST000175') & 
        (f.col('variantId') =='1_184051811_G_A') &
        (f.col('tagVariantId') =='1_184041593_C_A')
    )
    .drop('chromosome')
    .show() 
)

+----------+---------------+---------------+---------+-----------------+-------------+--------------------+--------------------+
|   studyId|      variantId|   tagVariantId|R_overall|          pics_mu|pics_postprob|pics_95_perc_credset|pics_99_perc_credset|
+----------+---------------+---------------+---------+-----------------+-------------+--------------------+--------------------+
|GCST000175|1_184051811_G_A|1_184041593_C_A| 0.966352|7.561187580349171|          NaN|                true|                true|
+----------+---------------+---------------+---------+-----------------+-------------+--------------------+--------------------+



In [11]:
(
    ld_data
    .filter(
        (f.col('studyId') =='GCST000175') & 
        (f.col('variantId') =='1_184051811_G_A') &
        (f.col('tagVariantId') =='1_184041593_C_A')
    )
    # .distinct()
    .select('variantId','gnomadPopulation','studyId','sampleSize','relativeSampleSize', 'tagVariantId', 'r')
    .show() 
)

+---------------+----------------+----------+----------+------------------+---------------+------------------+
|      variantId|gnomadPopulation|   studyId|sampleSize|relativeSampleSize|   tagVariantId|                 r|
+---------------+----------------+----------+----------+------------------+---------------+------------------+
|1_184051811_G_A|             nfe|GCST000175|   30968.0|               1.0|1_184041593_C_A|0.9663520794409783|
+---------------+----------------+----------+----------+------------------+---------------+------------------+



In [28]:
from scipy.stats import norm


def _pics_standard_deviation(neglog_p: Column, r: Column, k: float) -> Column:
    """Compute the PICS standard deviation.

    Args:
        neglog_p (Column): Negative log p-value
        r (Column): R-squared
        k (float): Empiric constant that can be adjusted to fit the curve, 6.4 recommended.

    Returns:
        Column: PICS standard deviation

    Examples:
        >>> k = 6.4
        >>> d = [(1.0, 1.0), (10.0, 1.0), (10.0, 0.5), (100.0, 0.5), (1.0, 0.0)]
        >>> spark.createDataFrame(d).toDF("neglog_p", "r").withColumn("std", _pics_standard_deviation(f.col("neglog_p"), f.col("r"), k)).show()
        +--------+---+-----------------+
        |neglog_p|  r|              std|
        +--------+---+-----------------+
        |     1.0|1.0|              0.0|
        |    10.0|1.0|              0.0|
        |    10.0|0.5|1.571749395040553|
        |   100.0|0.5|4.970307999319905|
        |     1.0|0.0|              0.5|
        +--------+---+-----------------+
        <BLANKLINE>
    """
    return f.sqrt(1 - f.abs(r) ** k) * f.sqrt(neglog_p) / 2


def _pics_mu(neglog_p: Column, r: Column) -> Column:
    """Compute the PICS mu.

    Args:
        neglog_p (Column): Negative log p-value
        r (Column): R

    Returns:
        Column: PICS mu

    Examples:
        >>> d = [(1.0, 1.0), (10.0, 1.0), (10.0, 0.5), (100.0, 0.5), (1.0, 0.0)]
        >>> spark.createDataFrame(d).toDF("neglog_p", "r").withColumn("mu", _pics_mu(f.col("neglog_p"), f.col("r"))).show()
        +--------+---+----+
        |neglog_p|  r|  mu|
        +--------+---+----+
        |     1.0|1.0| 1.0|
        |    10.0|1.0|10.0|
        |    10.0|0.5| 2.5|
        |   100.0|0.5|25.0|
        |     1.0|0.0| 0.0|
        +--------+---+----+
        <BLANKLINE>
    """
    return neglog_p * (r**2)


def _neglog_p(p_value_mantissa: Column, p_value_exponent: Column) -> Column:
    """Compute the negative log p-value.

    Args:
        p_value_mantissa (Column): P-value mantissa
        p_value_exponent (Column): P-value exponent

    Returns:
        Column: Negative log p-value

    Examples:
        >>> d = [(1, 1), (5, -2), (1, -1000)]
        >>> df = spark.createDataFrame(d).toDF("p_value_mantissa", "p_value_exponent")
        >>> df.withColumn("neg_log_p", _neglog_p(f.col("p_value_mantissa"), f.col("p_value_exponent"))).show()
        +----------------+----------------+------------------+
        |p_value_mantissa|p_value_exponent|         neg_log_p|
        +----------------+----------------+------------------+
        |               1|               1|              -1.0|
        |               5|              -2|1.3010299956639813|
        |               1|           -1000|            1000.0|
        +----------------+----------------+------------------+
        <BLANKLINE>
    """
    return -1 * (f.log10(p_value_mantissa) + p_value_exponent)


def _weighted_r_overall(
    chromosome: Column,
    study_id: Column,
    variant_id: Column,
    tag_variant_id: Column,
    relative_sample_size: Column,
    r: Column,
) -> Column:
    """Aggregation of weighted R information using ancestry proportions.

    Args:
        chromosome (Column): Chromosome
        study_id (Column): Study identifier
        variant_id (Column): Variant identifier
        tag_variant_id (Column): Tag variant identifier
        relative_sample_size (Column): Relative sample size
        r (Column): Correlation

    Returns:
        Column: Estimates weighted R information
    """
    pseudo_r = f.when(r >= 1, 0.9999995).otherwise(r)
    zscore_overall = f.sum(f.atan(pseudo_r) * relative_sample_size).over(
        Window.partitionBy(chromosome, study_id, variant_id, tag_variant_id)
    )
    return f.round(f.tan(zscore_overall), 6)


@f.udf(t.DoubleType())
def _norm_sf(mu: float, std: float, neglog_p: float):
    """Returns the survival function of the normal distribution for the p-value.

    Args:
        mu (float): mean
        std (float): standard deviation
        neglog_p (float): negative log p-value

    Returns:
        float: survival function

    Examples:
        >>> d = [{"mu": 0, "neglog_p": 0, "std": 1}, {"mu": 1, "neglog_p": 10, "std": 10}]
        >>> spark.createDataFrame(d).withColumn("norm_sf", _norm_sf(f.col("mu"), f.col("std"), f.col("neglog_p"))).show()
        +---+--------+---+-------------------+
        | mu|neglog_p|std|            norm_sf|
        +---+--------+---+-------------------+
        |  0|       0|  1|                1.0|
        |  1|      10| 10|0.36812025069351895|
        +---+--------+---+-------------------+
        <BLANKLINE>
    """
    try:
        return float(norm(mu, std).sf(neglog_p) * 2)
    except TypeError:
        return None


def _pics_posterior_probability(
    pics_mu: Column,
    pics_std: Column,
    neglog_p: Column,
    chromosome: Column,
    study_id: Column,
    variant_id: Column,
) -> Column:
    """Compute the PICS posterior probability.

    Args:
        pics_mu (Column): PICS mu
        pics_std (Column): PICS standard deviation
        neglog_p (Column): Negative log p-value
        chromosome (Column): Chromosome column
        study_id (Column): Study ID column
        variant_id (Column): Variant ID column

    Returns:
        Column: PICS posterior probability
    """
    pics_relative_prob = f.when(pics_std == 0, 1.0).otherwise(
        _norm_sf(pics_mu, pics_std, neglog_p)
    )
    w_lead = Window.partitionBy(chromosome, study_id, variant_id)
    pics_relative_prob_sum = f.sum(pics_relative_prob).over(w_lead)
    return pics_relative_prob / pics_relative_prob_sum


def _is_in_credset(
    chromosome: Column,
    study_id: Column,
    variant_id: Column,
    pics_postprob: Column,
    credset_probability: float,
) -> Column:
    """Check whether a variant is in the XX% credible set.

    Args:
        chromosome (Column): Chromosome column
        study_id (Column): Study ID column
        variant_id (Column): Variant ID column
        pics_postprob (Column): PICS posterior probability column
        credset_probability (float): Credible set probability

    Returns:
        Column: Whether the variant is in the credible set
    """
    w_cumlead = (
        Window.partitionBy(chromosome, study_id, variant_id)
        .orderBy(f.desc(pics_postprob))
        .rowsBetween(Window.unboundedPreceding, Window.currentRow)
    )
    pics_postprob_cumsum = f.sum(pics_postprob).over(w_cumlead)
    w_credset = Window.partitionBy(chromosome, study_id, variant_id).orderBy(
        pics_postprob_cumsum
    )
    return f.when(
        f.lag(pics_postprob_cumsum, 1).over(w_credset) >= credset_probability, False
    ).otherwise(True)


k = 6.3

# (
#     ld_data
#     .filter(
#         (f.col('studyId') =='GCST000175') & 
#         (f.col('variantId') =='1_184051811_G_A') &
#         (f.col('tagVariantId') =='1_184041593_C_A')
#     )
#     # .distinct()
#     .select("chromosome", 'variantId','gnomadPopulation','studyId','sampleSize','relativeSampleSize', 'tagVariantId', 'r', "pValueMantissa", "pValueExponent")
#     .withColumn(
#         "R_overall",
#         _weighted_r_overall(
#             f.col("chromosome"),
#             f.col("studyId"),
#             f.col("variantId"),
#             f.col("tagVariantId"),
#             f.col("relativeSampleSize"),
#             f.col("r"),
#         ),
#     )
#     .withColumn(
#         "pics_mu",
#         _pics_mu(
#             _neglog_p(f.col("pValueMantissa"), f.col("pValueExponent")),
#             f.col("R_overall"),
#         ),
#     )
#     .withColumn(
#         "pics_std",
#         _pics_standard_deviation(
#             _neglog_p(f.col("pValueMantissa"), f.col("pValueExponent")),
#             f.col("R_overall"),
#             k,
#         ),
#     )
#     .withColumn(
#         "pics_postprob",
#         _pics_posterior_probability(
#             f.col("pics_mu"),
#             f.col("pics_std"),
#             _neglog_p(f.col("pValueMantissa"), f.col("pValueExponent")),
#             f.col("chromosome"),
#             f.col("studyId"),
#             f.col("variantId"),
#         ),
#     )
#     .withColumn(
#         "pics_95_perc_credset",
#         _is_in_credset(
#             f.col("chromosome"),
#             f.col("studyId"),
#             f.col("variantId"),
#             f.col("pics_postprob"),
#             0.95,
#         ),
#     )
#     .withColumn(
#         "pics_99_perc_credset",
#         _is_in_credset(
#             f.col("chromosome"),
#             f.col("studyId"),
#             f.col("variantId"),
#             f.col("pics_postprob"),
#             0.99,
#         ),
#     )
#     .show() 
# )


## Prototyping PICS again
**Input:**
- associations: DataFrame,
- studies: DataFrame,

In [10]:
# studies = spark.read.parquet()
ld_data.count()

52676571

In [12]:
ld_data = ld_data.select("chromosome", 'variantId','gnomadPopulation','studyId','sampleSize','relativeSampleSize', 'tagVariantId', 'r', "pValueMantissa", "pValueExponent").distinct()

In [3]:
(
    ld_data
    # .distinct()
    .select("chromosome", 'variantId','gnomadPopulation','studyId','sampleSize','relativeSampleSize', 'tagVariantId', 'r', "pValueMantissa", "pValueExponent")
    .distinct()
    .withColumn(
        "R_overall",
        _weighted_r_overall(
            f.col("chromosome"),
            f.col("studyId"),
            f.col("variantId"),
            f.col("tagVariantId"),
            f.col("relativeSampleSize"),
            f.col("r"),
        ),
    )
    .withColumn(
        "pics_mu",
        _pics_mu(
            _neglog_p(f.col("pValueMantissa"), f.col("pValueExponent")),
            f.col("R_overall"),
        ),
    )
    .withColumn(
        "pics_std",
        _pics_standard_deviation(
            _neglog_p(f.col("pValueMantissa"), f.col("pValueExponent")),
            f.col("R_overall"),
            k,
        ),
    )
    .withColumn(
        "pics_postprob",
        _pics_posterior_probability(
            f.col("pics_mu"),
            f.col("pics_std"),
            _neglog_p(f.col("pValueMantissa"), f.col("pValueExponent")),
            f.col("chromosome"),
            f.col("studyId"),
            f.col("variantId"),
        ),
    )
    .withColumn(
        "pics_95_perc_credset",
        _is_in_credset(
            f.col("chromosome"),
            f.col("studyId"),
            f.col("variantId"),
            f.col("pics_postprob"),
            0.95,
        ),
    )
    .withColumn(
        "pics_99_perc_credset",
        _is_in_credset(
            f.col("chromosome"),
            f.col("studyId"),
            f.col("variantId"),
            f.col("pics_postprob"),
            0.99,
        ),
    )
    .write.mode('overwrite').parquet('gs://genetics_etl_python_playground/XX.XX/output/python_etl/parquet/pics_credible_set_update')
)
print('done')

done


In [8]:
assoc_columns = [
    "chromosome",
    "position",
    "referenceAllele",
    "alternateAllele",
    "variantId",
    "studyId",
    "pValueMantissa",
    "pValueExponent",
    "beta",
    "beta_ci_lower",
    "beta_ci_upper",
    "odds_ratio",
    "odds_ratio_ci_lower",
    "odds_ratio_ci_upper",
    "qualityControl",
]
# studies = etl.spark.read.parquet(cfg.etl.gwas_ingest.outputs.gwas_catalog_studies)


spark.read.parquet('gs://genetics_etl_python_playground/XX.XX/output/python_etl/parquet/gwas_catalog_associations').select(*assoc_columns).filter(f.size(f.col('qualityControl'))>1).show(4, False, True)

-RECORD 0-----------------------------------------------------------
 chromosome          | 11                                           
 position            | 99622442                                     
 referenceAllele     | C                                            
 alternateAllele     | G                                            
 variantId           | 11_99622442_C_G                              
 studyId             | GCST000109                                   
 pValueMantissa      | 9.0                                          
 pValueExponent      | -6                                           
 beta                | null                                         
 beta_ci_lower       | null                                         
 beta_ci_upper       | null                                         
 odds_ratio          | null                                         
 odds_ratio_ci_lower | null                                         
 odds_ratio_ci_upper | null       

In [6]:
df = spark.read.parquet('gs://genetics_etl_python_playground/XX.XX/output/python_etl/parquet/pics_credible_set').persist()
df.show()

22/12/15 11:05:20 WARN org.apache.spark.sql.execution.CacheManager: Asked to cache already cached data.


+----------+--------------+------------+--------+---------------+---------------+--------------+--------------+------+--------------------+--------------------+----------+-------------------+-------------------+--------------------+----------+------------+---------+-------+--------+-------------+--------------------+--------------------+
|chromosome|     variantId|     studyId|position|referenceAllele|alternateAllele|pValueMantissa|pValueExponent|  beta|       beta_ci_lower|       beta_ci_upper|odds_ratio|odds_ratio_ci_lower|odds_ratio_ci_upper|      qualityControl|sampleSize|tagVariantId|R_overall|pics_mu|pics_std|pics_postprob|pics_95_perc_credset|pics_99_perc_credset|
+----------+--------------+------------+--------+---------------+---------------+--------------+--------------+------+--------------------+--------------------+----------+-------------------+-------------------+--------------------+----------+------------+---------+-------+--------+-------------+--------------------+--

In [10]:
df.filter(~f.isnan(f.col('pics_postprob')) & f.col('pics_postprob').isNotNull()).count()

1467587

In [11]:
df.filter(f.col('pics_99_perc_credset') | f.col('pics_95_perc_credset')).count()

1270796

In [12]:
df.count()

35685656

In [16]:
df.filter(f.col('pics_99_perc_credset') | f.col('pics_95_perc_credset')).select(f.col('variantId')).distinct().count()

231664

In [19]:
df.filter(f.col('tagVariantId').isNotNull()).select(f.col('variantId'), 'tagVariantId').distinct().count()

17095414

In [34]:
test = (
    df
    .filter((f.col('studyId')=='GCST000263') & (f.col("variantId")=="1_10293054_T_C"))
    .select('pValueMantissa','chromosome', 'pValueExponent', 'studyId', 'variantId', 'tagVariantId', 'R_overall','pics_mu', 'pics_std', 'pics_postprob', 'pics_95_perc_credset', 'pics_99_perc_credset')
    .distinct()
    .persist()
)

test.show(1, False, True)

-RECORD 0----------------------------------
 pValueMantissa       | 2.0                
 chromosome           | 1                  
 pValueExponent       | -12                
 studyId              | GCST000289         
 variantId            | 1_62465961_C_A     
 tagVariantId         | 1_62664760_C_T     
 R_overall            | -0.946715          
 pics_mu              | 10.485427553848778 
 pics_std             | 0.9298549512727865 
 pics_postprob        | NaN                
 pics_95_perc_credset | false              
 pics_99_perc_credset | false              
only showing top 1 row



In [26]:
df.filter(f.col('pics_95_perc_credset') & (f.col('pics_postprob')>0) & (~f.isnan(f.col('pics_postprob')))).show(1, False, True)


-RECORD 0----------------------------------
 chromosome           | 1                  
 variantId            | 1_19816373_C_T     
 studyId              | GCST000311         
 position             | 19816373           
 referenceAllele      | C                  
 alternateAllele      | T                  
 pValueMantissa       | 7.0                
 pValueExponent       | -9                 
 beta                 | null               
 beta_ci_lower        | null               
 beta_ci_upper        | null               
 odds_ratio           | null               
 odds_ratio_ci_lower  | null               
 odds_ratio_ci_upper  | null               
 qualityControl       | []                 
 sampleSize           | 42300.0            
 tagVariantId         | 1_19816373_C_T     
 R_overall            | 1.0                
 pics_mu              | 8.154901959985743  
 pics_std             | 0.0                
 pics_postprob        | 0.1852548353298165 
 pics_95_perc_credset | true    

In [37]:
pocok_notna = (
    test
    # .distinct()
    # .select("chromosome", 'variantId','gnomadPopulation','studyId','sampleSize','relativeSampleSize', 'tagVariantId', 'r', "pValueMantissa", "pValueExponent")
    .drop('pics_mu', 'pics_std', 'pics_postprob', 'pics_95_perc_credset', 'pics_99_perc_credset')
    # .distinct()
    # .withColumn(
    #     "R_overall",
    #     _weighted_r_overall(
    #         f.col("chromosome"),
    #         f.col("studyId"),
    #         f.col("variantId"),
    #         f.col("tagVariantId"),
    #         f.col("relativeSampleSize"),
    #         f.col("r"),
    #     ),
    # )
    .withColumn(
        "pics_mu_recal",
        _pics_mu(
            _neglog_p(f.col("pValueMantissa"), f.col("pValueExponent")),
            f.col("R_overall"),
        ),
    )
    .withColumn(
        "pics_std_recal",
        _pics_standard_deviation(
            _neglog_p(f.col("pValueMantissa"), f.col("pValueExponent")),
            f.col("R_overall"),
            k,
        ),
    )
    .filter(~f.isnan(f.col('pics_std_recal')))
    .withColumn(
        "pics_postprob_recal",
        _pics_posterior_probability(
            f.col("pics_mu_recal"),
            f.col("pics_std_recal"),
            _neglog_p(f.col("pValueMantissa"), f.col("pValueExponent")),
            f.col("chromosome"),
            f.col("studyId"),
            f.col("variantId"),
        ),
    )
    .withColumn(
        "pics_95_perc_credset_recal",
        _is_in_credset(
            f.col("chromosome"),
            f.col("studyId"),
            f.col("variantId"),
            f.col("pics_postprob_recal"),
            0.95,
        ),
    )
    .withColumn(
        "pics_99_perc_credset_recal",
        _is_in_credset(
            f.col("chromosome"),
            f.col("studyId"),
            f.col("variantId"),
            f.col("pics_postprob_recal"),
            0.99,
        ),
    )
    # .write.mode('overwrite').parquet('gs://genetics_etl_python_playground/XX.XX/output/python_etl/parquet/pics_credible_set_update')
    .persist()
)

pocok.show(1, False, True)

-RECORD 0----------------------------------------
 pValueMantissa             | 1.0                
 chromosome                 | 1                  
 pValueExponent             | -8                 
 studyId                    | GCST003496         
 variantId                  | 1_43610348_G_A     
 tagVariantId               | 1_43649351_A_G     
 R_overall                  | 0.72293            
 pics_mu_pocok              | 4.1810222792       
 pics_std_pocok             | 1.3194613210926271 
 pics_postprob_pocok        | 1.0                
 pics_95_perc_credset_pocok | true               
 pics_99_perc_credset_pocok | true               
only showing top 1 row



In [48]:
x = (
    test.drop('pValueMantissa','chromosome', 'pValueExponent')
    .join(
        pocok.drop('pValueMantissa','chromosome', 'pValueExponent', 'R_overall'),
        on=['studyId', 'variantId', 'tagVariantId'],
        how='left'
    )
    # .filter(f.col('pics_postprob') != f.col('pics_postprob_pocok'))
    .persist()
)

colnames = x.columns
colnames.sort()

22/12/15 16:29:49 WARN org.apache.spark.sql.execution.CacheManager: Asked to cache already cached data.


In [52]:
x.select(*[
    f.col(c).alias(c.replace('pocok', 'recal')) for c in colnames
]).show(1, False, True)

-RECORD 0----------------------------------------
 R_overall                  | -0.946715          
 pics_95_perc_credset       | false              
 pics_95_perc_credset_recal | true               
 pics_99_perc_credset       | false              
 pics_99_perc_credset_recal | true               
 pics_mu                    | 10.485427553848778 
 pics_mu_recal              | 10.485427553848778 
 pics_postprob              | NaN                
 pics_postprob_recal        | 1.0                
 pics_std                   | 0.9298549512727865 
 pics_std_recal             | 0.9237525075131878 
 studyId                    | GCST000289         
 tagVariantId               | 1_62664760_C_T     
 variantId                  | 1_62465961_C_A     
only showing top 1 row



In [55]:
df.unpersist()
df = spark.read.parquet('gs://genetics_etl_python_playground/XX.XX/output/python_etl/parquet/pics_credible_set').persist()
df.show()

+----------+-----------------+-------------+--------+---------------+---------------+--------------+--------------+-------+--------------------+--------------------+----------+-------------------+-------------------+--------------------+----------+------------+---------+-------+--------+-------------+--------------------+--------------------+
|chromosome|        variantId|      studyId|position|referenceAllele|alternateAllele|pValueMantissa|pValueExponent|   beta|       beta_ci_lower|       beta_ci_upper|odds_ratio|odds_ratio_ci_lower|odds_ratio_ci_upper|      qualityControl|sampleSize|tagVariantId|R_overall|pics_mu|pics_std|pics_postprob|pics_95_perc_credset|pics_99_perc_credset|
+----------+-----------------+-------------+--------+---------------+---------------+--------------+--------------+-------+--------------------+--------------------+----------+-------------------+-------------------+--------------------+----------+------------+---------+-------+--------+-------------+--------

In [56]:
df.filter(~f.isnan(f.col('pics_postprob')) & f.col('pics_postprob').isNotNull()).count()

1467587

In [4]:

df = spark.read.parquet('gs://genetics_etl_python_playground/XX.XX/output/python_etl/parquet/pics_credible_set').persist()
df.show()

+----------+-----------------+-------------+--------+---------------+---------------+--------------+--------------+-------+--------------------+--------------------+----------+-------------------+-------------------+--------------------+----------+------------+---------+-------+--------+-------------+--------------------+--------------------+
|chromosome|        variantId|      studyId|position|referenceAllele|alternateAllele|pValueMantissa|pValueExponent|   beta|       beta_ci_lower|       beta_ci_upper|odds_ratio|odds_ratio_ci_lower|odds_ratio_ci_upper|      qualityControl|sampleSize|tagVariantId|R_overall|pics_mu|pics_std|pics_postprob|pics_95_perc_credset|pics_99_perc_credset|
+----------+-----------------+-------------+--------+---------------+---------------+--------------+--------------+-------+--------------------+--------------------+----------+-------------------+-------------------+--------------------+----------+------------+---------+-------+--------+-------------+--------

In [5]:
(
    df
    .count()
)

35626446

In [9]:
(
    df
    .filter(~f.isnan(f.col('pics_postprob')) & f.col('pics_postprob').isNotNull())
    # .show(1, False, True)
    .count()
)

35475806

In [10]:
(
    df
    .filter(~f.isnan(f.col('pics_postprob')) & f.col('pics_postprob').isNotNull())
    .select('studyId', 'variantId', 'tagVariantId')
    .distinct()
    # .show(1, False, True)
    .count()
)

30235987

In [13]:
(
    df
    .filter(~f.isnan(f.col('pics_postprob')) & f.col('pics_postprob').isNotNull() & f.col('pics_99_perc_credset'))
    .select('studyId', 'variantId', 'tagVariantId')
    .distinct()
    # .show(1, False, True)
    .count()
)

18686423

In [16]:
from scipy.stats import norm
def _pval_to_zscore(pval_col: Column) -> Column:
    """Convert p-value column to z-score column.

    Args:
        pval_col (Column): pvalues to be casted to floats.

    Returns:
        Column: p-values transformed to z-scores

    Examples:
        >>> d = [{"id": "t1", "pval": "1"}, {"id": "t2", "pval": "0.9"}, {"id": "t3", "pval": "0.05"}, {"id": "t4", "pval": "1e-300"}, {"id": "t5", "pval": "1e-1000"}, {"id": "t6", "pval": "NA"}]
        >>> df = spark.createDataFrame(d)
        >>> df.withColumn("zscore", _pval_to_zscore(f.col("pval"))).show()
        +---+-------+----------+
        | id|   pval|    zscore|
        +---+-------+----------+
        | t1|      1|       0.0|
        | t2|    0.9|0.12566137|
        | t3|   0.05|  1.959964|
        | t4| 1e-300| 37.537838|
        | t5|1e-1000| 37.537838|
        | t6|     NA|      null|
        +---+-------+----------+
        <BLANKLINE>

    """
    pvalue_float = pval_col.cast(t.FloatType())
    pvalue_nozero = f.when(pvalue_float == 0, sys.float_info.min).otherwise(
        pvalue_float
    )
    return f.udf(
        lambda pv: float(abs(norm.ppf((float(pv)) / 2))) if pv else None,
        t.FloatType(),
    )(pvalue_nozero)



In [17]:
d = [{"id": "t1", "pval": "1"}, {"id": "t2", "pval": "0.9"}, {"id": "t3", "pval": "0.05"}, {"id": "t4", "pval": "1e-300"}, {"id": "t5", "pval": "1e-1000"}, {"id": "t6", "pval": "NA"}]
df = spark.createDataFrame(d)
df.withColumn("zscore", _pval_to_zscore(f.col("pval"))).show()

+---+-------+----------+
| id|   pval|    zscore|
+---+-------+----------+
| t1|      1|       0.0|
| t2|    0.9|0.12566137|
| t3|   0.05|  1.959964|
| t4| 1e-300| 37.537838|
| t5|1e-1000| 37.537838|
| t6|     NA|      null|
+---+-------+----------+



In [19]:
def _get_reverse_complement(allele_col: Column) -> Column:
    """A function to return the reverse complement of an allele column.

    It takes a string and returns the reverse complement of that string if it's a DNA sequence,
    otherwise it returns the original string

    Args:
        allele_col (Column): The column containing the allele to reverse complement.

    Returns:
        A column that is the reverse complement of the allele column.

    Examples:
        >>> d = [{"allele": 'A'}, {"allele": 'T'},{"allele": 'G'}, {"allele": 'C'},{"allele": 'AC'}, {"allele": 'GT'},{"allele": '?'}, {"allele": None}]
        >>> df = spark.createDataFrame(d)
        >>> df.withColumn("revcom_allele", _get_reverse_complement(f.col("allele"))).show()
        +------+-------------+
        |allele|revcom_allele|
        +------+-------------+
        |     A|            T|
        |     T|            A|
        |     G|            C|
        |     C|            G|
        |    AC|           GT|
        |    GT|           AC|
        |     ?|            ?|
        |  null|         null|
        +------+-------------+
        <BLANKLINE>

    """
    allele_col = f.upper(allele_col)
    return f.when(
        allele_col.rlike("[ACTG]+"),
        f.reverse(f.translate(allele_col, "ACTG", "TGAC")),
    ).otherwise(allele_col)


d = [{"allele": 'A'}, {"allele": 'T'},{"allele": 'G'}, {"allele": 'C'},{"allele": 'AC'}, {"allele": 'GTaatc'},{"allele": '?'}, {"allele": None}]
df = spark.createDataFrame(d)
df.withColumn("revcom_allele", _get_reverse_complement(f.col("allele"))).show()

+------+-------------+
|allele|revcom_allele|
+------+-------------+
|     A|            T|
|     T|            A|
|     G|            C|
|     C|            G|
|    AC|           GT|
|GTaatc|       GATTAC|
|     ?|            ?|
|  null|         null|
+------+-------------+



In [25]:
def _harmonize_beta(
    effect_size: Column, confidence_interval: Column, needs_harmonization: Column
) -> Column:
    """A function to harmonize beta.

    If the confidence interval contains the word "increase" and the effect size needs to be harmonized,
    then multiply the effect size by -1

    Args:
        effect_size (Column): The effect size column from the dataframe
        confidence_interval (Column): The confidence interval of the effect size.
        needs_harmonization (Column): a boolean column that indicates whether the effect size needs to be flipped

    Returns:
        A column of the harmonized beta values.

    Examples:
        >>> d = [
        >>>     {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9] unit decrease', 'needs_harmonization': True},
        >>>     {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9] unit decrease', 'needs_harmonization': False},
        >>>     {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9] unit increase', 'needs_harmonization': True},
        >>>     {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9] unit increase', 'needs_harmonization': False},
        >>>     {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9]', 'needs_harmonization': False},
        >>> ]
        >>> df = spark.createDataFrame(d)
        >>> df.withColumn("beta", _harmonize_beta(f.col("effect_size"), f.col('confidence_interval'), f.col('needs_harmonization'))).show()
        +--------------------+-----------+-------------------+----+
        | confidence_interval|effect_size|needs_harmonization|beta|
        +--------------------+-----------+-------------------+----+
        |[0.1-0.9] unit de...|        0.6|               true| 0.6|
        |[0.1-0.9] unit de...|        0.6|              false|-0.6|
        |[0.1-0.9] unit in...|        0.6|               true|-0.6|
        |[0.1-0.9] unit in...|        0.6|              false| 0.6|
        |           [0.1-0.9]|        0.6|              false|null|
        +--------------------+-----------+-------------------+----+
        <BLANKLINE>

    """
    # The effect is given as beta, if the confidence interval contains 'increase' or 'decrease'
    beta = f.when(
        confidence_interval.contains("increase")
        | confidence_interval.contains("decrease"),
        effect_size,
    )
    # Flipping beta if harmonization is required or effect negated by saying 'decrease'
    return f.when(
        (confidence_interval.contains("increase") & needs_harmonization)
        | (confidence_interval.contains("decrease") & ~needs_harmonization),
        beta * -1,
    ).otherwise(beta)

d = [
    {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9] unit decrease', 'needs_harmonization': True},
    {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9] unit decrease', 'needs_harmonization': False},
    {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9] unit increase', 'needs_harmonization': True},
    {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9] unit increase', 'needs_harmonization': False},
    {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9]', 'needs_harmonization': False},
]
df = spark.createDataFrame(d)
df.withColumn("beta", _harmonize_beta(f.col("effect_size"), f.col('confidence_interval'), f.col('needs_harmonization'))).show()

+--------------------+-----------+-------------------+----+
| confidence_interval|effect_size|needs_harmonization|beta|
+--------------------+-----------+-------------------+----+
|[0.1-0.9] unit de...|        0.6|               true| 0.6|
|[0.1-0.9] unit de...|        0.6|              false|-0.6|
|[0.1-0.9] unit in...|        0.6|               true|-0.6|
|[0.1-0.9] unit in...|        0.6|              false| 0.6|
|           [0.1-0.9]|        0.6|              false|null|
+--------------------+-----------+-------------------+----+



In [27]:
def _harmonize_odds_ratio(
    effect_size: Column, confidence_interval: Column, needs_harmonization: Column
) -> Column:
    """Harmonizing odds ratio.

    Args:
        effect_size (Column): The effect size column from the dataframe
        confidence_interval (Column): The confidence interval of the effect size.
        needs_harmonization (Column): a boolean column that indicates whether the odds ratio needs to be flipped

    Returns:
        A column with the odds ratio, or 1/odds_ratio if harmonization required.
    
    
    
    """
    # The confidence interval tells if we are not dealing with betas -> OR
    odds_ratio = f.when(
        ~confidence_interval.contains("increase")
        & ~confidence_interval.contains("decrease"),
        effect_size,
    )

    return f.when(needs_harmonization, 1 / odds_ratio).otherwise(odds_ratio)


df = spark.createDataFrame(d)
df.withColumn("odds_ratio", _harmonize_odds_ratio(f.col("effect_size"), f.col('confidence_interval'), f.col('needs_harmonization'))).show()

+--------------------+-----------+-------------------+----------+
| confidence_interval|effect_size|needs_harmonization|odds_ratio|
+--------------------+-----------+-------------------+----------+
|[0.1-0.9] unit de...|        0.6|               true|      null|
|[0.1-0.9] unit de...|        0.6|              false|      null|
|[0.1-0.9] unit in...|        0.6|               true|      null|
|[0.1-0.9] unit in...|        0.6|              false|      null|
|           [0.1-0.9]|        0.6|              false|       0.6|
+--------------------+-----------+-------------------+----------+



In [28]:
d = [
    {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9] unit decrease', 'needs_harmonization': True},
    {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9]', 'needs_harmonization': False},
    {"effect_size": 0.6, 'confidence_interval': '[0.1-0.9]', 'needs_harmonization': True},
]
df = spark.createDataFrame(d)
df.withColumn("odds_ratio", _harmonize_odds_ratio(f.col("effect_size"), f.col('confidence_interval'), f.col('needs_harmonization'))).show()

+--------------------+-----------+-------------------+------------------+
| confidence_interval|effect_size|needs_harmonization|        odds_ratio|
+--------------------+-----------+-------------------+------------------+
|[0.1-0.9] unit de...|        0.6|               true|              null|
|           [0.1-0.9]|        0.6|              false|               0.6|
|           [0.1-0.9]|        0.6|               true|1.6666666666666667|
+--------------------+-----------+-------------------+------------------+



In [31]:
def _calculate_beta_ci(beta: Column, zscore: Column, direction: Column) -> Column:
    """Calculating confidence intervals for beta values.

    Args:
        beta (Column): The beta value for the given row
        zscore (Column): The z-score of the beta coefficient.
        direction (Column): This is the direction of the confidence interval. It can be either "upper" or "lower".

    Returns:
        The upper and lower bounds of the confidence interval for the beta coefficient.

    Examples:
        >>> d = [
        ...    {"beta": 0.6, 'zscore': 3, 'direction': 'upper'},
        ...    {"beta": 0.6, 'zscore': 3, 'direction': 'lower'},
        ...    {"beta": 0.6, 'zscore': 3, 'direction': 'something'},
        ...    {"beta": None, 'zscore': 3, 'direction': 'lower'},
        ... ]
        >>> df = spark.createDataFrame(d)
        >>> df.withColumn("beta",  _calculate_beta_ci(f.col("beta"), f.col('zscore'), f.col('direction'))).show()
        +----+---------+------+------------------------+
        |beta|direction|zscore|beta_confidence_interval|
        +----+---------+------+------------------------+
        | 0.6|    upper|     3|                   0.992|
        | 0.6|    lower|     3|     0.20800000000000002|
        | 0.6|somethign|     3|                    null|
        |null|    lower|     3|                    null|
        +----+---------+------+------------------------+
        <BLANKLINE>

    """
    zscore_95 = f.lit(1.96)
    return (
        f.when(direction == "upper", beta + f.abs(zscore_95 * beta) / zscore)
        .when(direction == "lower", beta - f.abs(zscore_95 * beta) / zscore)
        .otherwise(None)
    )

d = [
    {"beta": 0.6, 'zscore': 3, 'direction': 'upper'},
    {"beta": 0.6, 'zscore': 3, 'direction': 'lower'},
    {"beta": 0.6, 'zscore': 3, 'direction': 'something'},
    {"beta": None, 'zscore': 3, 'direction': 'lower'},
]
df = spark.createDataFrame(d)
df.withColumn("beta_confidence_interval", _calculate_beta_ci(f.col("beta"), f.col('zscore'), f.col('direction'))).show()


+----+---------+------+------------------------+
|beta|direction|zscore|beta_confidence_interval|
+----+---------+------+------------------------+
| 0.6|    upper|     3|                   0.992|
| 0.6|    lower|     3|     0.20800000000000002|
| 0.6|something|     3|                    null|
|null|    lower|     3|                    null|
+----+---------+------+------------------------+



In [32]:
def _calculate_or_ci(odds_ratio: Column, zscore: Column, direction: Column) -> Column:
    """Calculating confidence intervals for odds-ratio values.

    Args:
        odds_ratio (Column): The odds ratio of the association between the exposure and outcome.
        zscore (Column): The z-score of the confidence interval.
        direction (Column): This is either "upper" or "lower" and determines whether the upper or lower confidence interval is calculated.

    Returns:
        The upper and lower bounds of the 95% confidence interval for the odds ratio.

    Examples:
        >>> d = [
        ...     {"odds_ratio": 0.6, 'zscore': 3, 'direction': 'upper'},
        ...     {"odds_ratio": 1.6, 'zscore': 3, 'direction': 'lower'},
        ...     {"odds_ratio": 0.6, 'zscore': 3, 'direction': 'something'},
        ...     {"odds_ratio": None, 'zscore': 3, 'direction': 'lower'},
        ... ]
        >>> df = spark.createDataFrame(d)
        >>> df.withColumn("or_confidence_interval", _calculate_or_ci(f.col("odds_ratio"), f.col('zscore'), f.col('direction'))).show()
        +---------+----------+------+----------------------+
        |direction|odds_ratio|zscore|or_confidence_interval|
        +---------+----------+------+----------------------+
        |    upper|       0.6|     3|    0.8377075574145849|
        |    lower|       1.6|     3|    1.1769597039688107|
        |something|       0.6|     3|                  null|
        |    lower|      null|     3|                  null|
        +---------+----------+------+----------------------+
        <BLANKLINE>

    """
    zscore_95 = f.lit(1.96)
    odds_ratio_estimate = f.log(odds_ratio)
    odds_ratio_se = odds_ratio_estimate / zscore
    return f.when(
        direction == "upper",
        f.exp(odds_ratio_estimate + f.abs(zscore_95 * odds_ratio_se)),
    ).when(
        direction == "lower",
        f.exp(odds_ratio_estimate - f.abs(zscore_95 * odds_ratio_se)),
    )


d = [
    {"odds_ratio": 0.6, 'zscore': 3, 'direction': 'upper'},
    {"odds_ratio": 1.6, 'zscore': 3, 'direction': 'lower'},
    {"odds_ratio": 0.6, 'zscore': 3, 'direction': 'something'},
    {"odds_ratio": None, 'zscore': 3, 'direction': 'lower'},
]
df = spark.createDataFrame(d)
df.withColumn("or_confidence_interval", _calculate_or_ci(f.col("odds_ratio"), f.col('zscore'), f.col('direction'))).show()


+---------+----------+------+----------------------+
|direction|odds_ratio|zscore|or_confidence_interval|
+---------+----------+------+----------------------+
|    upper|       0.6|     3|    0.8377075574145849|
|    lower|       1.6|     3|    1.1769597039688107|
|something|       0.6|     3|                  null|
|    lower|      null|     3|                  null|
+---------+----------+------+----------------------+



In [53]:
    data = [
        # Beta, no need to harmonization:
        (
            "A",
            "C",
            "C",
            5,
            -11,
            0.6,
            "[0.013-0.024] unit increase",
            True,
            True,
            False,
            False,
            False,
        ),
        # Beta, harmonization required:
        (
            "A",
            "C",
            "T",
            5,
            -11,
            0.6,
            "[0.013-0.024] unit increase",
            True,
            False,
            False,
            False,
            False,
        ),
        # Beta, effect dropped due to palindromic alleles:
        (
            "A",
            "T",
            "A",
            5,
            -11,
            0.6,
            "[0.013-0.024] unit increase",
            False,
            False,
            False,
            False,
            True,
        ),
    ]
    mock_df = (
        spark.createDataFrame(
            data,
            [
                "referenceAllele",
                "alternateAllele",
                "riskAllele",
                "pValueMantissa",
                "pValueExponent",
                "effectSize",
                "confidenceInterval",
                "isBeta",
                "isBetaPositive",
                "isOR",
                "isORPositive",
                "isPalindrome",
            ],
        )
        .withColumn("qualityControl", f.array())
        .persist()
    )
mock_df.show()

+---------------+---------------+----------+--------------+--------------+----------+--------------------+------+--------------+-----+------------+------------+--------------+
|referenceAllele|alternateAllele|riskAllele|pValueMantissa|pValueExponent|effectSize|  confidenceInterval|isBeta|isBetaPositive| isOR|isORPositive|isPalindrome|qualityControl|
+---------------+---------------+----------+--------------+--------------+----------+--------------------+------+--------------+-----+------------+------------+--------------+
|              A|              C|         C|             5|           -11|       0.6|[0.013-0.024] uni...|  true|          true|false|       false|       false|            []|
|              A|              C|         T|             5|           -11|       0.6|[0.013-0.024] uni...|  true|         false|false|       false|       false|            []|
|              A|              T|         A|             5|           -11|       0.6|[0.013-0.024] uni...| false|       

In [54]:
mock_df.show()

+---------------+---------------+----------+--------------+--------------+----------+--------------------+------+--------------+-----+------------+------------+--------------+
|referenceAllele|alternateAllele|riskAllele|pValueMantissa|pValueExponent|effectSize|  confidenceInterval|isBeta|isBetaPositive| isOR|isORPositive|isPalindrome|qualityControl|
+---------------+---------------+----------+--------------+--------------+----------+--------------------+------+--------------+-----+------------+------------+--------------+
|              A|              C|         C|             5|           -11|       0.6|[0.013-0.024] uni...|  true|          true|false|       false|       false|            []|
|              A|              C|         T|             5|           -11|       0.6|[0.013-0.024] uni...|  true|         false|false|       false|       false|            []|
|              A|              T|         A|             5|           -11|       0.6|[0.013-0.024] uni...| false|       

In [67]:
def _check_rsids(gnomad: Column, gwas: Column) -> Column:
    """If the intersection of the two arrays is greater than 0, return True, otherwise return False.

    Args:
        gnomad (Column): rsids from gnomad
        gwas (Column): rsids from the GWAS Catalog

    Returns:
        A boolean column that is true if the GnomAD rsIDs can be found in the GWAS rsIDs.
    
    Examples:
        >>> d = [
        ...    (1, ["rs123", "rs523"], ["rs123"], True),
        ...    (2, [], ["rs123"], False),
        ...    (2, ["rs123", "rs523"], [], False),
        ... ]
        >>> df = spark.createDataFrame(d, ['associationId', 'gnomad', 'gwas', 'isRsIdMatched'])
        >>> df.withColumn("or_confidence_interval", _check_rsids(f.col("gnomad"),f.col('gwas'))).show()
        +-------------+--------------+-------+-------------+----------------------+
        |associationId|        gnomad|   gwas|isRsIdMatched|or_confidence_interval|
        +-------------+--------------+-------+-------------+----------------------+
        |            1|[rs123, rs523]|[rs123]|         true|                  true|
        |            2|            []|[rs123]|        false|                 false|
        |            2|[rs123, rs523]|     []|        false|                 false|
        +-------------+--------------+-------+-------------+----------------------+
        <BLANKLINE>

    """
    return f.when(f.size(f.array_intersect(gnomad, gwas)) > 0, True).otherwise(False)


d = [
    (1, ["rs123", "rs523"], ["rs123"]),
    (2, [], ["rs123"]),
    (3, ["rs123", "rs523"], []),
    (4, [], []),
]
df = spark.createDataFrame(d, ['associationId', 'gnomad', 'gwas'])
df.withColumn("or_confidence_interval", _check_rsids(f.col("gnomad"),f.col('gwas'))).show()


+-------------+--------------+-------+----------------------+
|associationId|        gnomad|   gwas|or_confidence_interval|
+-------------+--------------+-------+----------------------+
|            1|[rs123, rs523]|[rs123]|                  true|
|            2|            []|[rs123]|                 false|
|            3|[rs123, rs523]|     []|                 false|
|            4|            []|     []|                 false|
+-------------+--------------+-------+----------------------+



In [70]:

def _check_concordance(
    risk_allele: Column, reference_allele: Column, alternate_allele: Column
) -> Column:
    """A function to check if the risk allele is concordant with the alt or ref allele.

    If the risk allele is the same as the reference or alternate allele, or if the reverse complement of
    the risk allele is the same as the reference or alternate allele, then the allele is concordant.
    If no mapping is available (ref/alt is null), the function returns True.

    Args:
        risk_allele (Column): The allele that is associated with the risk of the disease.
        reference_allele (Column): The reference allele from the GWAS catalog
        alternate_allele (Column): The alternate allele of the variant.

    Returns:
        A boolean column that is True if the risk allele is the same as the reference or alternate allele,
        or if the reverse complement of the risk allele is the same as the reference or alternate allele.
    
    Examples:
        >>> d = [
        ...     ('A', 'A', 'G'),
        ...     ('A', 'T', 'G'),
        ...     ('A', 'C', 'G'),
        ...     ('A', 'A', '?'),
        ...     (None, None, 'A'),
        ... ]
        >>> df = spark.createDataFrame(d, ['riskAllele', 'referenceAllele', 'alternateAllele'])
        >>> df.withColumn("isConcordant", _check_concordance(f.col("riskAllele"),f.col('referenceAllele'), f.col('alternateAllele'))).show()
        +-------------+--------------+-------+-------------+----------------------+
        |associationId|        gnomad|   gwas|isRsIdMatched|or_confidence_interval|
        +-------------+--------------+-------+-------------+----------------------+
        |            1|[rs123, rs523]|[rs123]|         true|                  true|
        |            2|            []|[rs123]|        false|                 false|
        |            2|[rs123, rs523]|     []|        false|                 false|
        +-------------+--------------+-------+-------------+----------------------+
        <BLANKLINE>

    """
    # Calculating the reverse complement of the risk allele:
    risk_allele_reverse_complement = f.when(
        risk_allele.rlike(r"^[ACTG]+$"),
        f.reverse(f.translate(risk_allele, "ACTG", "TGAC")),
    ).otherwise(risk_allele)

    # OK, is the risk allele or the reverse complent is the same as the mapped alleles:
    return (
        f.when(
            (risk_allele == reference_allele) | (risk_allele == alternate_allele),
            True,
        )
        # If risk allele is found on the negative strand:
        .when(
            (risk_allele_reverse_complement == reference_allele)
            | (risk_allele_reverse_complement == alternate_allele),
            True,
        )
        # If risk allele is ambiguous, still accepted: < This condition could be reconsidered
        .when(risk_allele == "?", True)
        # If the association could not be mapped we keep it:
        .when(reference_allele.isNull(), True)
        # Allele is discordant:
        .otherwise(False)
    )


d = [
    ('A', 'A', 'G'),
    ('A', 'T', 'G'),
    ('A', 'C', 'G'),
    ('A', 'A', '?'),
    (None, None, 'A'),
]
df = spark.createDataFrame(d, ['riskAllele', 'referenceAllele', 'alternateAllele'])
df.withColumn("isConcordant", _check_concordance(f.col("riskAllele"),f.col('referenceAllele'), f.col('alternateAllele'))).show()


+----------+---------------+---------------+------------+
|riskAllele|referenceAllele|alternateAllele|isConcordant|
+----------+---------------+---------------+------------+
|         A|              A|              G|        true|
|         A|              T|              G|        true|
|         A|              C|              G|       false|
|         A|              A|              ?|        true|
|      null|           null|              A|        true|
+----------+---------------+---------------+------------+



In [71]:
def _find_mappings_to_drop(association_id: Column, filter_column: Column) -> Column:
    """Flagging mappings to drop for each association.

    Some associations have multiple mappings. Some has matching rsId others don't. We only
    want to drop the non-matching mappings, when a matching is available for the given association.
    This logic can be generalised for other measures eg. allele concordance.

    Args:
        association_id (Column): association identifier column
        filter_column (Column): boolean col indicating to keep a mapping

    Returns:
        A column with a boolean value.

    Examples:
    >>> d = [
    ...    (1, False),
    ...    (1, False),
    ...    (2, False),
    ...    (2, True),
    ...    (3, True),
    ...    (3, True),
    ...]
    >>> df = spark.createDataFrame(d, ['associationId', 'filter'])
    >>> df.withColumn("isConcordant", _find_mappings_to_drop(f.col("associationId"),f.col('filter'))).show()
    +-------------+------+------------+
    |associationId|filter|isConcordant|
    +-------------+------+------------+
    |            1| false|        true|
    |            1| false|        true|
    |            3|  true|        true|
    |            3|  true|        true|
    |            2| false|       false|
    |            2|  true|        true|
    +-------------+------+------------+
    <BLANKLINE>
    
    """
    w = Window.partitionBy(association_id)

    # Generating a boolean column informing if the filter column contains true anywhere for the association:
    aggregated_filter = f.when(
        f.array_contains(f.collect_set(filter_column).over(w), True), True
    ).otherwise(False)

    # Generate a filter column:
    return f.when(aggregated_filter & (~filter_column), False).otherwise(True)


d = [
    (1, False),
    (1, False),
    (2, False),
    (2, True),
    (3, True),
    (3, True),
]
df = spark.createDataFrame(d, ['associationId', 'filter'])
df.withColumn("isConcordant", _find_mappings_to_drop(f.col("associationId"),f.col('filter'))).show()


+-------------+------+------------+
|associationId|filter|isConcordant|
+-------------+------+------------+
|            1| false|        true|
|            1| false|        true|
|            3|  true|        true|
|            3|  true|        true|
|            2| false|       false|
|            2|  true|        true|
+-------------+------+------------+



In [73]:
def _keep_mapping_with_top_maf(association_id: Column, maf_column: Column) -> Column:
    """For each association ID, keep the row with the highest MAF.

    Args:
        association_id (Column): Column
        maf_column (Column): the column that contains the MAF values

    Returns:
        A column with a boolean value.

    Examples:
    >>> d = [
    ...    (1, 0.4),
    ...    (1, 0.4),
    ...    (2, 0.2),
    ...    (2, 0.1),
    ...    (3, None),
    ... ]
    >>> df = spark.createDataFrame(d, ['associationId', 'maxMaf'])
    >>> df.withColumn("isTopMaf", _keep_mapping_with_top_maf(f.col("associationId"),f.col('maxMaf'))).show()
    +-------------+------+--------+
    |associationId|maxMaf|isTopMaf|
    +-------------+------+--------+
    |            1|   0.4|    true|
    |            1|   0.4|   false|
    |            3|  null|    true|
    |            2|   0.2|    true|
    |            2|   0.1|   false|
    +-------------+------+--------+
    <BLANKLINE>
    
    """
    w = Window.partitionBy(association_id).orderBy(f.desc(maf_column))
    row_numbers = f.row_number().over(w)
    return f.when(row_numbers == 1, True).otherwise(False)



d = [
    (1, 0.4),
    (1, 0.4),
    (2, 0.2),
    (2, 0.1),
    (3, None),
]
df = spark.createDataFrame(d, ['associationId', 'maxMaf'])
df.withColumn("isTopMaf", _keep_mapping_with_top_maf(f.col("associationId"),f.col('maxMaf'))).show()

+-------------+------+--------+
|associationId|maxMaf|isTopMaf|
+-------------+------+--------+
|            1|   0.4|    true|
|            1|   0.4|   false|
|            3|  null|    true|
|            2|   0.2|    true|
|            2|   0.1|   false|
+-------------+------+--------+

